# Comp-Comm data process

## Functions
- Read csv file:
    - File path: '{TPBench_Root}/result/log'
    - File name: 'np{np}_{kernel_name}-{run_mode}-{program_chunk}({counter_unit})_ntest{ntest}_N{N}.csv'
    - File format:
        - Row: rank0~rank{np-1} 
        - Col: step0~step{ntest-1}
- Compute data:
    - Compute communication delay from ns to us
    - Compute computation perforamce with GFLOPs unit and B/c unit
- Draw figures:
    - Draw histogram
    - Draw heatmap

## Realization

- Class `Chunk_Data`:
    - variable members: `kernel_name`(string), `np`(int), `run_mode`(string), `ntest`(int), `N`(int), `program_chunk`(string), `file_name`(string), `raw_data`(dataframe, int), `processed_data`(dataframe, float), `raw_unit`(string), `processed_unit`(string)
    - Functions:
        - `read_data(file_name, path)`: read file and update `raw_data`
        - `process_data()`: process `raw_data` (mode1: unit transfer; mode2: calculate performance value)
        - `draw_heatmap()`: draw heatmap of `processed_data` (x-axis:rank0~rank{np-1}, y-axis: step0~step{ntest-1})
        - `draw_histgram()`: draw histgram (mode 1: rank0~rank{np-1}'s mean\min\max\mid value, mode 2: step0~step{ntest-1}'s mean\min\max\mid value)

- Class `Kernel_Datas`: 
    - variable members: `np`(int), `run_mode`(string), `ntest`(int), `N`(int), `program_chunks`(list of string), `datas`(dictionary, {`program_chunk`: `data`(`Raw_Data` class)})
    - Functions:
        - `create_funtion(task_set, program_chunk)`: update parameters and add datas dictionary
        - `add_data()`: add a new (`program_chunk`, `data`) pair
        - `draw_figures()`: draw heatmap and draw histgrams of mode 1&2
        - `extract_task_set(task_set)`: 

- Class `Kernels`:
    - variable members: `task_set`(set of string, `task_name`(`np{np}_{kernel_name}-{run_mode}_ntest{ntest}_N{N}`)), `kernel_datas`(dictionary, `{task_name: data(Raw_Data)}`), `file_path`(string)
    - Functions:
        - `create_funtion(path)`: read all files in path and call `get_all_tasks()`
        - `get_all_tasks()`: read the file list from specific path and classify the kernel with same parameter(`kernel_name`, `np`, `run_mode`, `ntest`, `N`)
        - `extract_file_name(file_name)`: get `task_set` and `program_chunk` in `file_name`
        - `get_task_name(kernel_name, np, run_mode, ntest, N)`: input "`kernel_name`, `np`, `run_mode`, `ntest`, `N`" and return a string `task_name` in format `np{np}_{kernel_name}-{run_mode}_ntest{ntest}_N{N}`
        - `get_task_names(kernel_names, nps, run_modes, ntests, Ns)`: input "`kernel_names`, `nps`, `run_modes`, `ntests`, `Ns`" and return a list `task_names` in the `np{np}_{kernel_name}-{run_mode}_ntest{ntest}_N{N}` format
        - `add_kernel_data()`: input `task_name` and maintain `kernel_datas` dictionary:
            - Case "no `task_name` in `kernel_datas`": create a Kernel_Datas class and add this pair into `kernel_datas` dictionary 
            - Case "exist `task_name` in `kernel_datas`": add a new `program_chunk` in existed `Raw_Data` class
        - `draw_figures(task_names = list)`: draw figures of task in `taks_names` if task's `task_name` is in `task_set`

- `Main Function`:
    - variables: `kernel_names`(list of string), `nps`(list of int), `run_modes`(list of int),  `ntest` (list of int), `Ns`(list of int), `kernels`(class Kernels)
    - call `kernels.get_all_tasks()`
    - for `kernel_name`, `np`, `run_mode`, `ntest`, `N` :
        - call `task_name` = `kernels.get_task_name()`
        - call `kernels.draw_figures(task_name)`
        

In [ ]:
import tp_coll_data as TPcd
import os

if __name__ == '__main__':
    Current_DIR = os.getcwd()
    TPBench_DIR = os.path.dirname(os.path.dirname(Current_DIR))
    
    # results = ['result_4', 'result_1', 'result_2', 'result_3']
    results = ['result']

    for res in results:
        result_path = os.path.join(TPBench_DIR, res)
        log_path = os.path.join(result_path, 'log')
        kernels = TPcd.Kernels(log_path)
        # kernels.delete_invalid_files()
        nps = [64, 128]
        
        program_chunks = ['comm(ns)', 'GEMM(ns)', 'Jacobi(ns)']
        processed_unit_dict = {'comm(ns)': ['us', 'KB_per_sec'], 'GEMM(ns)':['us', 'KB_per_sec']}

        task_names = kernels.get_all_task_names()
        # kernels.draw_figures(task_names=task_names, program_chunks=program_chunks, processed_units_dict=processed_unit_dict, save_dir=result_path, save_mode='old', figure_mode='statistic')
        kernels.draw_figures(task_names=task_names, program_chunks=program_chunks, processed_units_dict=processed_unit_dict, save_dir=result_path, save_mode='old', figure_mode='fixed')
        kernels.draw_all_figures(task_names = task_names, program_chunks = program_chunks, processed_units_dict = processed_unit_dict, save_dir = result_path)
        kernels.generate_summary(result_path)

        kernel_names = ['gemmallreduce', 'gemmbcast', 'jacobi2d5p_sendrecv'] 
        kernel_names = ['gemmbcast'] 

        compute_units = ['avx2', 'avx512']
        program_patterns = ['commonly', 'compcomm']
        sockets = ['']
        
        # run_modes = ['avx2_compcomm', 'twosocket-commonly', 'twosocket-compcomm', 'onesocket-commonly', 'onesocket-compcomm']
        # run_modes = ['commonly']
        ntests = [100]
        # Ns = [256, 512, 1024, 200, 500, 1000, 2000]
        Ns = [512]

        mpi_funcs=['allreduce', 'bcast']
        mpi_func_algs = {
            'allreduce': ['fixed', 'basicLinear', 'nonoverlapping', 'recursiveDoubling', 'ring', 'segmentedRing', 'rabenseifner'],
            'bcast': ['fixed', 'basicLinear', 'chain', 'pipeline', 'splitBinaryTree', 'binaryTree', 'binomial', 'knomial', 'scatterAllgather', 'scatterAllgatherRing']
        }

        mpi_funcs=['bcast']
        mpi_func_algs = {
            'allreduce': ['rabenseifner'],
            'bcast': ['fixed']
        }
    
        program_chunks = ['comm(ns)', 'GEMM(cy)']
        processed_unit_dict = {'comm(ns)': ['us'], 'GEMM(cy)':['bytes_per_cycle']}
        run_modes = TPcd.Run_Modes.generate_run_modes(mode=2, program_patterns=program_patterns, compute_units=compute_units, sockets=sockets, mpi_funcs=mpi_funcs, mpi_func_algs=mpi_func_algs)
        task_names = kernels.get_task_names(kernel_names=kernel_names, nps=nps, run_modes=run_modes, ntests=ntests, Ns=Ns)
        # print(run_modes)
        # print(task_names)
        # kernels.draw_figures(task_names=task_names, program_chunks=program_chunks, processed_units_dict=processed_unit_dict, save_dir=result_path, save_mode='old', figure_mode='interactive')


In [ ]:
import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import socket

def compare_figures(common_files, common_dirs, save_dir): 
    for j, file in enumerate(common_files):
        # if j > 3:
        #     break
        if file.endswith('.png'):
            fig, axes = plt.subplots(1, len(common_dirs), figsize=(22*len(common_dirs), 16))
            file_name = file.split('.')[0]
            for i, common_dir in enumerate(common_dirs):
                img = plt.imread(os.path.join(common_dir, file))
                axes[i].imshow(img)
                axes[i].axis('off')
            plt.tight_layout()
            plt.savefig(os.path.join(save_dir, file_name + '.png'))
            # plt.close()
            # plt.savefig(os.path.join(save_dir, file_name + '.svg'))
            plt.close()


if __name__ == '__main__':
    hostname = socket.gethostname()
    machine = None
    if "csprn1" in hostname:
        machine = "spr"
    elif "amd9654" in hostname:
        machine = "amd9654"
    elif "chgn1" in hostname:
        machine = "chgn1"
    elif "amd9754" in hostname:
        machine = "amd9754"
    if machine is None:
        exit("Invalid hostname.")

    result_dirs = ['result_1', 'result_2', 'result_3']

    Current_DIR = os.getcwd()
    TPBench_DIR = os.path.dirname(Current_DIR)

    # 获取文件夹路径
    histogram_dirs = []
    heatmap_dirs = []

    for result_dir in result_dirs:
        result_path = os.path.join(TPBench_DIR, result_dir)
        local_histogram_dirs = []
        local_heatmap_dirs = []
        for d in os.listdir(result_path):
            if os.path.isdir(os.path.join(result_path, d)):
                if d.startswith(f'{machine}-') and d.endswith('result-histogram'):
                    local_histogram_dirs.append(d)
                elif d.startswith(f'{machine}-') and d.endswith('result-heatmap'):
                    local_heatmap_dirs.append(d)
        local_histogram_dirs = sorted(local_histogram_dirs, key=lambda x: x.split('-')[1], reverse=True)[0]
        local_heatmap_dirs = sorted(local_heatmap_dirs, key=lambda x: x.split('-')[1], reverse=True)[0]
        histogram_dirs.append(os.path.join(result_path, local_histogram_dirs))
        heatmap_dirs.append(os.path.join(result_path, local_heatmap_dirs))

    print(histogram_dirs)
    print(heatmap_dirs)

    common_heatmap=set()
    common_histogram=set()

    for i, heatmap_dir in enumerate(heatmap_dirs):
        files = os.listdir(heatmap_dir)
        common_heatmap = common_heatmap.union(set(files))

    for i, histogram_dir in enumerate(histogram_dirs):
        files = os.listdir(histogram_dir)
        common_histogram = common_histogram.union(set(files))

    # time_stampe = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')

    save_dir = os.path.join(Current_DIR, f'{machine}-compare-{result_dirs[0]}-{result_dirs[-1]}-histogram')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    compare_figures(common_histogram, histogram_dirs, save_dir)


    save_dir = os.path.join(Current_DIR, f'{machine}-compare-{result_dirs[0]}-{result_dirs[-1]}-heatmap')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    compare_figures(common_heatmap, heatmap_dirs, save_dir)